## 🌌 Latent Space Ideation with Advanced Swarm Queen Optimization

This Google Colab notebook implements a powerful **latent space ideation framework** that combines the strengths of modern language models and swarm intelligence. It allows users to **generate novel, high-quality ideas** by navigating and manipulating embedding spaces using a custom-built Swarm Queen Optimization (SQO) mechanism.

### 🔍 What This Notebook Does

* **Encodes ideas into embeddings** using `sentence-transformers`
* **Explores latent space** via a swarm of intelligent agents guided by a central "Swarm Queen"
* **Interpolates and mutates concepts** in vector space to spark new innovations
* **Decodes ideas back to human language** using a large generative language model (e.g., `google/gemma-2b-it`)
* **Scores and ranks ideas** with a placeholder evaluator (easily replaceable with an LLM-based judge)

### 🧠 Key Components

* `TextEncoder`: Encodes text into latent space embeddings
* `SwarmAgent`: Generates novel idea vectors using guided interpolation and noise
* `SwarmQueen`: Maintains global guidance vector and adapts based on feedback
* `Projector`: Neural mapper to align encoder and decoder vector dimensions
* `LatentToTextDecoder`: Generates human-readable text from latent vectors
* `run_latent_sqo_pipeline()`: Orchestrates the entire pipeline

### 🧪 Real-World Use Case

Combines:

> 🧠 *"A smart water bottle that tracks hydration"*
> with
> ☀️ *"A solar-powered portable charger"*

To discover:

> 💡 *"A solar-powered smart water bottle that tracks hydration and purifies water using sunlight."*

### ⚙️ Runtime Instructions

1. **Change Runtime Type:**
   In the Colab menu, go to **`Runtime` → `Change runtime type`** and select **GPU (T4 or A100 preferred)**.

2. **Run Cells in Order:**
   Start from installing dependencies, then run each component step-by-step.

3. **Set Your HuggingFace Token (Optional):**
   If loading private models, securely pass your token using:

   ```python
   from google.colab import userdata  
   HF_TOKEN = userdata.get('Hugging')  
   ```

### 🧩 Plug-and-Play:

* Easily swap out encoder or decoder models (e.g., OpenRouter or local LLaMA variants).
* Add a real evaluator by integrating GPT-4 or Claude 3.5 with scoring prompts.



In [ ]:
from huggingface_hub import login
from google.colab import userdata

# Retrieve your secret API key
HF_TOKEN = userdata.get('Hugging')

# Log in to Hugging Face
login(token=HF_TOKEN)

In [ ]:
# Install necessary libraries
# pip install sentence-transformers numpy

from sentence_transformers import SentenceTransformer
import numpy as np

class LatentSpaceIdeator:
    """
    A simple implementation of the latent space ideation framework.
    This class can encode text ideas into embeddings, combine them,
    and then would typically use a decoder to generate a new idea.
    """

    def __init__(self, encoder_model_name='all-MiniLM-L6-v2'):
        """
        Initializes the ideator with a sentence-transformer model.

        Args:
            encoder_model_name (str): The name of the pre-trained model to use for encoding.
        """
        print("Loading the encoder model... This may take a moment.")
        # The encoder model turns text into meaningful numerical vectors (embeddings)
        self.encoder = SentenceTransformer(encoder_model_name)
        print("Encoder model loaded successfully.")

    def encode(self, text):
        """
        Encodes a single piece of text into a latent space vector.

        Args:
            text (str): The text to encode.

        Returns:
            np.ndarray: A numpy array representing the text in the latent space.
        """
        # The .encode() method creates the vector representation of the text.
        return self.encoder.encode(text)

    def interpolate(self, vec1, vec2, weight=0.5):
        """
        Creates a new vector by interpolating between two existing vectors.
        This is the core of the 'combining' process in the latent space.

        Args:
            vec1 (np.ndarray): The first vector.
            vec2 (np.ndarray): The second vector.
            weight (float): The weight for interpolation. 0.5 gives an even mix.
                            Closer to 0 leans towards vec1, closer to 1 leans towards vec2.

        Returns:
            np.ndarray: The new, interpolated vector.
        """
        # Linear interpolation: new_vector = (1 - weight) * vec1 + weight * vec2
        return (1 - weight) * vec1 + weight * vec2

    def find_most_similar(self, target_vector, candidate_texts):
        """
        A simple 'decoder' that finds the most similar text to a target vector.
        In a full implementation (like the paper), this would be a generative
        language model that generates text *from* the vector. For this example,
        we find the closest match from a predefined list.

        Args:
            target_vector (np.ndarray): The vector to decode.
            candidate_texts (list): A list of texts to compare against.

        Returns:
            str: The candidate text most similar to the target vector.
        """
        candidate_vectors = self.encoder.encode(candidate_texts)

        # Calculate cosine similarity between the target and all candidates
        # The formula is: (A . B) / (||A|| * ||B||)
        dot_products = np.dot(candidate_vectors, target_vector)
        norm_target = np.linalg.norm(target_vector)
        norm_candidates = np.linalg.norm(candidate_vectors, axis=1)

        similarities = dot_products / (norm_target * norm_candidates)

        # Find the index of the highest similarity score
        best_match_index = np.argmax(similarities)

        return candidate_texts[best_match_index]


# --- Real-World Use Case: Generating a New Tech Product ---

if __name__ == "__main__":
    # 1. Initialize the ideator
    ideator = LatentSpaceIdeator()

    # 2. Define the 'Problem' and 'Variable' ideas we want to combine
    problem_idea = "A smart water bottle that tracks hydration and reminds you to drink."
    variable_idea = "A portable solar charger for electronic devices."

    print(f"\nCombining Idea 1: '{problem_idea}'")
    print(f"Combining Idea 2: '{variable_idea}'")

    # 3. Encode these ideas into the latent space
    problem_vector = ideator.encode(problem_idea)
    variable_vector = ideator.encode(variable_idea)

    # 4. Explore the latent space by interpolating between the two ideas
    # This creates a new vector representing a blend of the two concepts.
    new_idea_vector = ideator.interpolate(problem_vector, variable_vector, weight=0.5)
    print("\nGenerated a new vector in the latent space by combining the two ideas.")

    # 5. Decode the new vector back into a human-readable idea.
    # For this example, we'll use a predefined list of potential product ideas
    # and find the one that is semantically closest to our new vector.
    # A more advanced system would use a generative LLM for this step.
    potential_solutions = [
        "A fitness tracker with a heart rate monitor.",
        "A self-cleaning water bottle with UV-C light.",
        "A water bottle that generates its own power from the sun to run smart features, like tracking intake and powering a purification system.",
        "A backpack with an integrated solar panel.",
        "A coffee mug that keeps your drink at the perfect temperature.",
        "A smart bottle cap that fits any standard bottle and tracks sips.",
        "A hydration pack for hiking that uses solar energy to cool the water."
    ]

    print("\nSearching for the closest concept in our 'solution space'...")

    generated_idea = ideator.find_most_similar(new_idea_vector, potential_solutions)

    # 6. Present the result
    print("\n--- New Product Idea ---")
    print(generated_idea)
    print("------------------------")
    print("\nThis idea was identified because its conceptual vector was closest to the 'average' of the smart water bottle and the solar charger vectors.")

In [ ]:
# ==============================================================================
# Latent Space Ideation with Advanced Swarm Queen Optimization (for Google Colab)
# ==============================================================================
#
# INSTRUCTIONS:
# 1. Open a new notebook in Google Colab (https://colab.research.google.com/).
# 2. In the menu, go to `Runtime` -> `Change runtime type` and select `T4 GPU` or another GPU.
#    This is crucial as the decoder model is large and will be very slow on a CPU.
# 3. Copy and paste the code from each section below into separate cells in your notebook.
# 4. Run the cells in order.
#
# NOTE: Loading the models, especially the decoder, will take several
# minutes and requires a significant amount of RAM.

# ===========================================
# CELL 1: Install Dependencies
# ===========================================
# Uncomment and run this cell to install the required libraries in your Colab environment.
# Using --upgrade ensures the latest library versions are installed.
# !pip install --upgrade transformers torch sentence-transformers accelerate bitsandbytes einops

# ===========================================
# CELL 2: Imports and Setup
# ===========================================
import torch
import torch.nn as nn
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM
from typing import List, Dict
import random
import numpy as np

# Set up device (use GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


# ===========================================
# CELL 3: Component 1 - Seed Encoder
# ===========================================
# This component encodes the initial text ideas into numerical vectors (embeddings).

class TextEncoder:
    """Encodes a list of text strings into latent space vectors."""
    def __init__(self, model_name='sentence-transformers/all-MiniLM-L6-v2', device=device):
        print(f"Loading encoder model: {model_name}...")
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name).to(device)
        self.model.eval()
        self.device = device
        print("Encoder loaded.")

    def encode(self, texts: List[str]) -> torch.Tensor:
        """Encodes a list of texts into a tensor of embeddings."""
        inputs = self.tokenizer(texts, return_tensors='pt', padding=True, truncation=True).to(self.device)
        with torch.no_grad():
            outputs = self.model(**inputs)
            embeddings = outputs.last_hidden_state.mean(dim=1)
        return embeddings

# ===========================================
# CELL 4: Component 2 - Advanced Latent Space Explorer (Swarm Queen)
# ===========================================
# This is an enhanced version inspired by the user's scheduling/logic engine.
# It includes a Queen that processes feedback and guides the swarm.

class SwarmAgent:
    """Represents an agent in the swarm that generates a new idea vector."""
    def __init__(self, aid: int, alpha: float = 0.7, noise_scale: float = 0.05):
        self.aid = aid
        self.alpha = alpha
        self.noise_scale = noise_scale

    def explore(self, queen_pool: torch.Tensor, peer_pool: torch.Tensor, queen_guidance: torch.Tensor) -> torch.Tensor:
        """
        Generates a new vector by combining influence from a queen, a peer, and global guidance.
        """
        queen = queen_pool[random.randint(0, len(queen_pool) - 1)]
        peer = peer_pool[random.randint(0, len(peer_pool) - 1)]

        # Move from peer towards the selected queen
        primary_direction = self.alpha * queen + (1 - self.alpha) * peer

        # Blend with the queen's global guidance vector
        guided_direction = 0.8 * primary_direction + 0.2 * queen_guidance

        # Add random noise for diversity
        noise = self.noise_scale * torch.randn_like(guided_direction)
        new_vector = guided_direction + noise
        return new_vector

class SwarmQueen:
    """
    The Queen processes feedback (scores) and maintains a global model to guide the swarm.
    """
    def __init__(self, num_agents: int, vector_dim: int):
        self.agent_weights = {i: 1.0 for i in range(num_agents)}
        self.global_model_vector = torch.zeros(vector_dim, device=device) # Represents promising directions
        self.previous_avg_score = 0.0

    def process_feedback(self, agent_vectors: Dict[int, torch.Tensor], scores: List[float]):
        """
        Updates agent weights and the global model based on the scores of the generated ideas.

        Args:
            agent_vectors: A dict mapping agent ID to the vector it produced.
            scores: A list of scores corresponding to the ideas from agent_vectors.
        """
        current_avg_score = sum(scores) / len(scores) if scores else 0
        reward = current_avg_score - self.previous_avg_score

        # Update agent weights based on reward
        for aid in self.agent_weights:
            if reward > 0:
                self.agent_weights[aid] = min(1.5, self.agent_weights[aid] * (1 + reward * 0.1))
            else:
                self.agent_weights[aid] = max(0.5, self.agent_weights[aid] * (1 + reward * 0.1))

        # Update the global model vector by moving towards high-scoring vectors
        self.global_model_vector *= 0.9 # Decay the old model
        for i, (aid, vector) in enumerate(agent_vectors.items()):
            # High-scoring vectors have more influence
            influence = scores[i] / 5.0 # Normalize score to 0-1 range
            weighted_influence = influence * self.agent_weights[aid]
            self.global_model_vector += 0.1 * weighted_influence * vector.to(device)

        # Normalize the global model vector to prevent its magnitude from exploding
        if torch.linalg.norm(self.global_model_vector) > 0:
            self.global_model_vector /= torch.linalg.norm(self.global_model_vector)

        self.previous_avg_score = current_avg_score

    def get_guidance(self) -> torch.Tensor:
        """Returns the current global guidance vector."""
        return self.global_model_vector

# ===========================================
# CELL 5: Component 3 - Cross-Modal Projector
# ===========================================
# This neural network maps vectors from the encoder's space to the decoder's space.

class Projector(nn.Module):
    """A simple MLP to project between embedding dimensions."""
    def __init__(self, input_dim, output_dim):
        super().__init__()
        self.mlp = nn.Sequential(
            nn.Linear(input_dim, (input_dim + output_dim) // 2),
            nn.ReLU(),
            nn.Linear((input_dim + output_dim) // 2, output_dim)
        )

    def forward(self, x):
        return self.mlp(x)

# ===========================================
# CELL 6: Component 4 - Decoder (Large Language Model)
# ===========================================
# This component generates human-readable text from a projected latent vector.

class LatentToTextDecoder:
    """Generates text from a latent vector using a causal language model."""
    def __init__(self, model_name='google/gemma-2b-it', device=device):
        print(f"Loading decoder model: {model_name}... (This will take time and memory)")
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token
        self.model = AutoModelForCausalLM.from_pretrained(
            model_name,
            torch_dtype=torch.float16,
            device_map='auto',
        )
        self.model.eval()
        print("Decoder loaded.")

    def generate(self, latent_vector: torch.Tensor, prompt_prefix: str = "Generate a creative, detailed idea based on this concept: ", max_length=60) -> str:
        """Generates text by feeding the latent vector as a continuous prompt."""
        embedding_layer = self.model.get_input_embeddings()
        prompt_tokens = self.tokenizer(prompt_prefix, return_tensors='pt').input_ids.to(latent_vector.device)
        prompt_embeds = embedding_layer(prompt_tokens)

        # Reshape the latent_vector to be 3D [batch, sequence, dim]
        # before concatenating with the 3D prompt_embeds.
        projected_embeds = latent_vector.unsqueeze(0).unsqueeze(0)

        inputs_embeds = torch.cat([prompt_embeds, projected_embeds], dim=1)

        with torch.no_grad():
            outputs = self.model.generate(
                inputs_embeds=inputs_embeds,
                max_new_tokens=max_length,
                do_sample=True,
                top_k=50,
                temperature=0.9,
                pad_token_id=self.tokenizer.eos_token_id
            )

        prompt_length = prompt_tokens.shape[1]
        generated_tokens = outputs[0][prompt_length:]
        return self.tokenizer.decode(generated_tokens, skip_special_tokens=True)

# ===========================================
# CELL 7: Component 5 - Evaluator (LLM-as-a-Judge)
# ===========================================
# This function scores generated ideas. We use a dummy version for demonstration.

def dummy_score_ideas(ideas: List[str]) -> List[float]:
    """Placeholder for an LLM-based evaluator. Assigns random scores."""
    print(f"Scoring {len(ideas)} ideas (using dummy scores)...")
    return [random.uniform(3.0, 5.0) for _ in ideas]

# ===========================================
# CELL 8: The Full Pipeline (Updated with Advanced SQO)
# ===========================================
# This function orchestrates the entire process.

def run_latent_sqo_pipeline(seed_ideas: List[str], iterations: int = 2, swarm_size: int = 20):
    """Runs the full ideation pipeline with the advanced Swarm Queen model."""
    # --- Initialization ---
    encoder_dim = 384
    decoder_dim = 2048

    encoder = TextEncoder(device=device)
    decoder = LatentToTextDecoder(device=device)
    projector = Projector(input_dim=encoder_dim, output_dim=decoder_dim).to(device).half()

    agents = [SwarmAgent(aid=i) for i in range(swarm_size)]
    queen = SwarmQueen(num_agents=swarm_size, vector_dim=encoder_dim)

    # --- Initial State ---
    print("\nEncoding initial seed ideas...")
    # *** FIX ***: Move the main vector storage to the CPU to avoid device mismatch errors.
    all_vectors = encoder.encode(seed_ideas).cpu()
    all_ideas = list(seed_ideas)
    all_scores = dummy_score_ideas(all_ideas)

    # --- Iterative Generation Loop ---
    for i in range(iterations):
        print(f"\n{'='*20} Iteration {i+1}/{iterations} {'='*20}")

        # 1. Identify Queens (top-k ideas)
        top_k = min(5, len(all_ideas))
        queen_indices = sorted(range(len(all_scores)), key=lambda k: all_scores[k], reverse=True)[:top_k]
        queen_pool = all_vectors[queen_indices]

        # 2. Explore: Each agent generates a new vector
        print("Swarm is exploring latent space...")
        agent_vectors = {}
        new_vectors_list = []
        # *** FIX ***: Move the guidance vector to the CPU before passing to agents.
        guidance_vector = queen.get_guidance().cpu()
        for agent in agents:
            new_vec = agent.explore(queen_pool, all_vectors, guidance_vector)
            agent_vectors[agent.aid] = new_vec
            new_vectors_list.append(new_vec)
        new_vectors = torch.stack(new_vectors_list)

        # 3. Project & Decode
        print("Projecting and decoding new vectors into ideas...")
        projected_vectors = projector(new_vectors.to(device).half())
        generated_ideas = [decoder.generate(vec) for vec in projected_vectors]
        for j, idea in enumerate(generated_ideas):
            print(f"  -> New Idea {j+1}: {idea.strip()}")

        # 4. Evaluate new ideas
        new_scores = dummy_score_ideas(generated_ideas)

        # 5. Queen processes feedback
        print("Queen is processing feedback...")
        queen.process_feedback(agent_vectors, new_scores)

        # 6. Update State
        all_ideas.extend(generated_ideas)
        # *** FIX ***: Concatenate tensors on the CPU.
        all_vectors = torch.cat([all_vectors, new_vectors], dim=0)
        all_scores.extend(new_scores)
        print(f"Population size is now {len(all_ideas)} ideas.")

    # --- Final Result ---
    sorted_ideas = [idea for _, idea in sorted(zip(all_scores, all_ideas), key=lambda pair: pair[0], reverse=True)]
    return sorted_ideas

# ===========================================
# CELL 9: Example Usage
# ===========================================
# Define your initial seed ideas and run the pipeline.

if __name__ == '__main__':
    seed_ideas = [
        "A drone that plants trees in deforested areas using biodegradable seed pods.",
        "A smart coffee mug that uses kinetic energy from stirring to keep the drink warm.",
        "Biodegradable shoes that contain seeds and can be planted at the end of their life.",
        "An AI therapist chatbot that specializes in helping people with social anxiety.",
        "A system for capturing and recycling greywater from showers for use in toilets."
    ]

    print("Starting Latent Space Ideation Pipeline with Advanced SQO...")
    final_ideas = run_latent_sqo_pipeline(seed_ideas, iterations=2, swarm_size=10)

    print("\n\n===================================")
    print("      Final Generated Ideas      ")
    print("   (Sorted by dummy score)   ")
    print("===================================")
    for i, idea in enumerate(final_ideas):
        print(f"{i+1}. {idea}")